# Experiments-Prepare

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [11]:
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.parsers as parsers
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.runner as runner
import source.players.detection as detection
import source.players.fabulous as fabulous

import source.util as util
import numpy as np
import pandas as pd
from importlib import *
import source.util as util
from copy import deepcopy, copy
import os

In [12]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(runner)
reload(parsers)
reload(game)
reload(environment)
reload(detection)
reload(fabulous)
reload(util)

<module 'source.util' from '/home/lorenzo/Scrivania/Polimi/Thesis/code/onlineSG/source/util.py'>

## Generating Targets

In [8]:
targets_dict = {i:[] for i in range(2,13)}
T = 15 #np.random.randint(2,10)
time_horizon =1000
while sum([len(targets_dict[x]) for x in range(2,13)]) < 9:
    targets = [round(x,3) for x in util.gen_norm_targets(T)]
    values = tuple((v, v) for v in targets)
    g = game.Game(values, time_horizon)
    g.attackers = [1]
    g.defenders = [0]
    s = util.support(g)
    if len(targets_dict[len(s)])==0:
        targets_dict[len(s)].append(s)

In [9]:
targets_dict

{2: [[0.991, 0.993]],
 3: [[0.942, 0.82, 0.909]],
 4: [[0.873, 0.966, 0.951, 0.738]],
 5: [[0.788, 0.939, 0.983, 0.713, 0.974]],
 6: [[0.781, 0.869, 0.91, 0.977, 0.87, 0.87]],
 7: [[0.828, 0.876, 0.973, 0.833, 0.809, 0.874, 0.961]],
 8: [[0.674, 0.688, 0.709, 0.79, 0.658, 0.979, 0.785, 0.697]],
 9: [[0.74, 0.752, 0.729, 0.797, 0.962, 0.999, 0.854, 0.78, 0.721]],
 10: [[0.716, 0.703, 0.719, 0.738, 0.793, 0.89, 0.698, 0.697, 0.886, 0.887]],
 11: [],
 12: []}

## Writing batch file

In [20]:
experiment_folder = "../Experiments/UnknownStochastic_Fabulous"
time_horizon = 1000
att_incl = True
defender = ["fabulous1"] #["br_mab1-1", "br_expert1-1"] #,"b2bw2w1","holmes1-1"]
#attacker = ["sta1"]
if not os.path.exists(experiment_folder):
    os.makedirs(experiment_folder)
for T in range(2,5,2):
    if not os.path.exists(experiment_folder + "/t" + str(T)):
        os.makedirs(experiment_folder + "/t" + str(T))
    targets = util.translate(targets_dict[T][0])
    mock_g = game.zs_game(targets, time_horizon)
    attacker = [str(attackers.StochasticAttacker(mock_g,1,1))]
    for P in range(1,4):
        profiles = util.gen_profiles(targets, [#(attackers.UnknownStochasticAttacker, 1),
                                               #(attackers.StackelbergAttacker, 1),
                                               (attackers.StochasticAttacker, P)])
        folder = experiment_folder + "/t" + str(T) + "/p" + str(P)
        if not os.path.exists(folder):
            os.makedirs(folder)
        batch_name = "_".join(defender)
        batch_file = batch_name + ".csv"
        batch_path = folder + "/" + batch_file
        with open(batch_path, "w+") as f:
            f.write(util.print_header(targets, len(profiles), att_incl))
            for a in attacker:
                for d in defender: 
                    f.write(util.gen_conf_row(d + "_vs_" + a.split('1')[0], time_horizon, targets, d, a,
                                              [str(p) for p in profiles], att_incl))        

In [19]:
rm -rf $experiment_folder

In [18]:
 ls $experiment_folder/t2/p3

fabulous.csv
